In [2]:
import numpy as np
import scipy as sp
import pandas as pd
import random
import matplotlib.pyplot as plt
import  networkx as nx
import tensorflow as tf

%matplotlib inline

In [3]:
def balanced_stochastic_blockmodel(communities=2, groupsize=3, p_in=1.0, p_out=0.0):
    #gives dense adjacency matrix representaiton of randomly generated SBM with balanced community size

    G = nx.planted_partition_graph(l=communities, k=groupsize, p_in=p_in, p_out =p_out)
    A = nx.adjacency_matrix(G).todense()
    
    return A


communities = 2 #number of communities, chance to 
group_size = 6 #number of nodes in each communitites (balanced so far)
dim_graph = communities*group_size
A = balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.45, p_out=0.01)
print A

data = [np.asarray(balanced_stochastic_blockmodel(communities=communities, groupsize=group_size, p_in=0.5, p_out=0.2)).astype(float) for i in range(10)]



[[0 1 1 0 1 1 0 0 0 0 0 0]
 [1 0 0 1 1 0 0 0 0 0 0 0]
 [1 0 0 0 0 1 0 0 0 0 0 0]
 [0 1 0 0 1 0 0 0 0 0 0 0]
 [1 1 0 1 0 0 0 0 0 0 0 0]
 [1 0 1 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 0 0]
 [0 0 0 0 0 0 0 0 1 0 1 1]
 [0 0 0 0 0 0 0 1 0 0 0 0]
 [0 0 0 0 0 0 0 0 0 0 1 0]
 [0 0 0 0 0 0 0 1 0 1 0 1]
 [0 0 0 0 0 0 0 1 0 0 1 0]]


In [4]:
communities = 6
Adj = tf.placeholder(tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float32),
                                      tf.ones([communities], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float32),
                                      tf.zeros([communities], dtype=tf.float32)])



eigenvec = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
#       sess.run(train)
        print sess.run([r, loss_grad, eigenvec_grad, loss], feed_dict = {Adj: data[i]})
    
    



[array([ 0.81813235]), [array([  1.01375779e-16])], [array([  4.31914899e-17])], 6.0]
[array([ 0.42968278]), [array([  3.49861110e-17])], [array([  2.64201234e-17])], 6.0]
[array([ 0.1773837]), [array([ 0.])], [array([  5.57006656e-17])], 6.0]
[array([-1.93327103]), [array([ 0.])], [array([  7.79677403e-18])], 6.0]
[array([-3.26667536]), [array([ -8.30498900e-18])], [array([ -4.15249450e-18])], 6.0]
[array([-0.91782256]), [array([ -5.12092146e-17])], [array([ -2.56046073e-17])], 6.0]
[array([ 2.48073006]), [array([ 0.])], [array([  9.91906307e-18])], 6.0]
[array([ 0.98609731]), [array([  4.68320317e-17])], [array([  2.34160159e-17])], 6.0]
[array([-0.05219224]), [array([  4.58670179e-17])], [array([  2.57508829e-17])], 6.0]
[array([ 2.22540444]), [array([ 0.])], [array([ 0.])], 6.0]


In [5]:
#now with training uncommentedout

In [7]:
communities = 6
Adj = tf.placeholder(tf.float64)

Diag = tf.diag(tf.reduce_sum(Adj,0))
Diag = tf.cast(Diag, tf.float64)

r =  tf.Variable(tf.random_normal(shape=[1], mean=0.0,
                                 stddev=2.0, dtype=tf.float64,
                                 seed=None, name=None))

Bethe_Hesse_neg = (tf.square(r)-1)*tf.diag(tf.ones(shape=[dim_graph], dtype=tf.float64))-tf.mul(r, Adj)+Diag 

eigenval, eigenvec = tf.self_adjoint_eig(Bethe_Hesse_neg)

true_assignment_a = tf.concat(0, [tf.zeros([communities], dtype=tf.float32),
                                      tf.ones([communities], dtype=tf.float32)])
true_assignment_b = tf.concat(0, [tf.ones([communities], dtype=tf.float32),
                                      tf.zeros([communities], dtype=tf.float32)])



eigenvec = tf.nn.softmax(tf.slice(eigenvec, [0,0], [dim_graph, 2]))#tf.argmax(tf.nn.softmax(tf.slice(Bethe_Hesse_neg, [0,0], [dim_graph, 2])), 1) #this is where we are supposed to take the eigenspace that corresponds to negative eigenvalues, however let's just learn this...
eigenvec = tf.cast(tf.reduce_sum(eigenvec, 1), dtype = tf.float32)


loss =  tf.minimum(tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_a))), 
              tf.reduce_sum(tf.square(tf.sub(eigenvec, true_assignment_b))))
#tf.reduce_sum(tf.square(tf.sub(c, true_assignment_a)))


optimizer = tf.train.GradientDescentOptimizer(0.001)
train = optimizer.minimize(loss)

eigenvec_grad = tf.gradients(eigenvec, r)
loss_grad = tf.gradients(loss, r)

init = tf.initialize_variables([r])

with tf.Session() as sess:
    for i in range(10):
        sess.run(init)
        print sess.run([r, loss_grad, eigenvec_grad, loss], feed_dict = {Adj: data[i]})
        sess.run(train)
        print sess.run([r, loss_grad, eigenvec_grad, loss], feed_dict = {Adj: data[i]})
    
    



[array([ 3.71714054]), [array([  5.37209992e-19])], [array([ -4.55276148e-19])], 6.0]


InvalidArgumentError: You must feed a value for placeholder tensor 'Placeholder_2' with dtype double
	 [[Node: Placeholder_2 = Placeholder[dtype=DT_DOUBLE, shape=[], _device="/job:localhost/replica:0/task:0/cpu:0"]()]]
Caused by op u'Placeholder_2', defined at:
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 162, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/Users/xiangli/anaconda/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/traitlets/config/application.py", line 596, in launch_instance
    app.start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/ipykernel/kernelapp.py", line 442, in start
    ioloop.IOLoop.instance().start()
  File "/Users/xiangli/anaconda/lib/python2.7/site-packages/zmq/eventloop/ioloop.py",